In [20]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import linregress
from itertools import product
import math
from sklearn.linear_model import LinearRegression
from dataclasses import dataclass


pd.options.plotting.backend = "plotly"

In [21]:
# https://docs.screeps.com/api/#Creep
CREEP_TICKS_LIFE = 1500
MAX_BODY_SIZE = 50
MAX_DISTANCE = 50

ROAD_ENERGY_REPAIR_RATIO = 0.01
ROAD_HEATH = 1000

TERRAIN_FACTOR = dict(plain=1, road=0.5)

# https://docs.screeps.com/api/#StructureRoad

In [22]:
class Proportions:
    """Not assuming one moving part for each carry part"""

    @classmethod
    def harvested_tick(cls, work):
        return work * 2

    @classmethod
    def capacity(cls, carry):
        return carry * 50

    @classmethod
    def ticks_to_full(cls, work, carry):
        return cls.capacity(carry) / cls.harvested_tick(work)

    @classmethod
    def ticks_per_move(cls, work, carry, move, direction="Forward"):
        return math.floor(work / move) if direction == "Forward" else math.floor((work + carry) / move)

    @classmethod
    def round_trip(cls, work, carry, move, distance):
        forward = distance * cls.ticks_per_move(work, carry, move, "Forward") + distance
        backwards = distance * cls.ticks_per_move(work, carry, move, "Backwards") + distance
        return (forward + backwards) + cls.ticks_to_full(work, carry) + 1

    @classmethod
    def body_cost(cls, work, carry, move):
        return work * 100 + carry * 50 + move * 50

    @classmethod
    def profit(cls, work, carry, move, distance):
        round_trips = round(CREEP_TICKS_LIFE / cls.round_trip(work, carry, distance, move))
        return round_trips * cls.capacity(carry) - cls.body_cost(work, carry, move)

    @classmethod
    def cost_benefit(cls, work, carry, move, distance):
        return cls.profit(work, carry, distance, move) / cls.body_cost(work, carry, move)
    
    @classmethod
    def cost_benefit_regression(cls):
        distances = [*range(5, MAX_DISTANCE, 1)]

        work_parts = [*range(1, MAX_BODY_SIZE)]
        carry_parts = [*range(1, MAX_BODY_SIZE)]
        move_parts = [*range(1, MAX_BODY_SIZE)]

        # Body combinations with less than MAX_BODY_SIZE parts
        parts_prod = filter(lambda x: (x[0] + x[1] + x[2]) < MAX_BODY_SIZE, product(work_parts, carry_parts, move_parts))
        parts_prod = filter(lambda x: cls.ticks_per_move(*x) < 10, parts_prod)
        # parts_prod

        cost_benefit_heatmap = list(map(lambda wc: [str(wc), *wc[0], wc[1], cls.cost_benefit(*wc[0], wc[1])], product(parts_prod, distances)))
        df = pd.DataFrame(cost_benefit_heatmap, columns=["label", "work_parts", "carry_parts", "walk_parts", "distance", "cost_benefit"])
        df = df[df['cost_benefit']>1]
        df['cost_benefit'] = round(df['cost_benefit'], 4)
        # print(df.groupby('distance').apply(lambda x: x.nlargest(5, 'cost_benefit')))

        df_reg = df.groupby('distance').apply(lambda x: x.nlargest(3, 'cost_benefit')).copy()
        df_reg = df_reg.reset_index(drop=True).sort_values(['distance', 'cost_benefit'], ascending=False)
        df_reg['ticks_per_move_loaded'] = (df_reg['work_parts'] + df_reg['carry_parts']) / df_reg['walk_parts']

        df_norm = df_reg.copy()

        df_norm['total_parts'] = df_norm['work_parts'] + df_norm['carry_parts'] + df_norm['walk_parts']
        df_norm['work_parts'] = df_norm['work_parts'] / df_norm['total_parts'] * 100
        df_norm['carry_parts'] = df_norm['carry_parts'] / df_norm['total_parts'] * 100
        df_norm['walk_parts'] = df_norm['walk_parts'] / df_norm['total_parts'] * 100

        df_norm.plot(x="distance", y=['work_parts', 'carry_parts', 'walk_parts'])

        fig = px.scatter(df_norm, x="distance", y=["work_parts", "carry_parts", "walk_parts"])

        return df_norm, fig


df_norm, fig = Proportions.cost_benefit_regression()
df_norm

,label,work_parts,carry_parts,walk_parts,distance,cost_benefit,ticks_per_move_loaded,total_parts
132,"((6, 18, 25), 49)",12.244898,36.734694,51.020408,49,1.5091,0.960000,49
133,"((7, 16, 24), 49)",14.893617,34.042553,51.063830,49,1.5000,0.958333,47
134,"((7, 16, 25), 49)",14.583333,33.333333,52.083333,49,1.4727,0.920000,48
129,"((7, 17, 25), 48)",14.285714,34.693878,51.020408,48,1.6250,0.960000,49
130,"((6, 18, 25), 48)",12.244898,36.734694,51.020408,48,1.5273,0.960000,49
...,...,...,...,...,...,...,...,...
4,"((11, 14, 13), 6)",28.947368,36.842105,34.210526,6,7.7143,1.923077,38
5,"((13, 14, 14), 6)",31.707317,34.146341,34.146341,6,7.7037,1.928571,41
0,"((16, 16, 17), 5)",32.653061,32.653061,34.693878,5,8.2923,1.882353,49
1,"((11, 12, 12), 5)",31.428571,34.285714,34.285714,5,8.2826,1.916667,35


In [23]:
fig.show()

In [42]:
class ProportionsRoads:

    @classmethod
    def harvested_tick(cls, work):
        return work * 2

    @classmethod
    def capacity(cls, carry):
        return carry * 50

    @classmethod
    def ticks_to_full(cls, work, carry):
        return cls.capacity(carry) / cls.harvested_tick(work)

    @classmethod
    def ticks_per_move(cls, work, carry, move, loaded=True, terrain_type="plain"):
        terrain_factor = TERRAIN_FACTOR[terrain_type]
        move_requirement = move / terrain_factor
        tpm = math.ceil((work + carry) / move_requirement) if loaded else math.ceil(work / move_requirement)
        return tpm or 1 # The game does not allow speeds greater than 1 move per tick

    @classmethod
    def round_trip(cls, work, carry, move, distance):
        forward = distance * cls.ticks_per_move(work, carry, move, "Forward") + distance
        backwards = distance * cls.ticks_per_move(work, carry, move, "Backwards") + distance
        return (forward + backwards) + cls.ticks_to_full(work, carry) + 1

    @classmethod
    def body_cost(cls, work, carry, move):
        return work * 100 + carry * 50 + move * 50

    @classmethod
    def road_repair_costs(cls, work, carry, move, distance, round_trip=True):
        body_size = work + carry + move

        if round_trip:
            body_size = body_size * 2

        road_decay_rate = 100 - body_size

        decay_cost_tick = ((CREEP_TICKS_LIFE / ROAD_HEATH) * road_decay_rate) * ROAD_ENERGY_REPAIR_RATIO

        fixed_cost = decay_cost_tick * distance
        return

    @classmethod
    def profit(cls, work, carry, move, distance):
        round_trips = round(CREEP_TICKS_LIFE / cls.round_trip(work, carry, distance, move))
        return round_trips * cls.capacity(carry) - cls.body_cost(work, carry, move)

    @classmethod
    def cost_benefit(cls, work, carry, move, distance):
        return cls.profit(work, carry, distance, move) / cls.body_cost(work, carry, move)

In [43]:
ProportionsRoads.ticks_per_move(work=10, carry=10, move=10, loaded=True, terrain_type="road")

1

In [47]:
ProportionsRoads.ticks_per_move(work=10, carry=10, move=10, loaded=True, terrain_type="plain")

2